# Exploring Variables #

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

df = pd.read_csv("../input/train.csv")
df.describe()

# Any results you write to the current directory are saved as output.

Obviously: each feature is present less than 10% of the time
X0 to X8 are text data

## Categorical Data vs Time ##

In [ ]:
categorical_indices = [0,1,2,3,4,5,6,8] 

for i in categorical_indices:
    print(df.groupby("X"+str(i))["X"+str(i)].count())

In [ ]:
sns.set_style("whitegrid")
for i in categorical_indices:
    plt.figure()
    ax = sns.boxplot(x="X"+str(i), y="y", data=df.loc[:,["X"+str(i),"y"]])

## Binary Variable Correlation ##

In [ ]:
#First, let's get actual column names
binary_indices = []

for i in range(9,385):
    if "X"+str(i) in df.columns.values:
        binary_indices.append("X"+str(i))
        
#I noticed later that X11 is 0 everywhere
binary_indices.remove("X11")


corr = df.loc[:,["y"] + binary_indices].corr()

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3,
            square=True, xticklabels=5, yticklabels=5,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)

In [ ]:
y_corr = corr.loc["X10":,"y"]
highly_correlated_variables = y_corr[y_corr.map(lambda x: abs(x) > 0.3)]
highly_correlated_variables = highly_correlated_variables.index
df.loc[:,highly_correlated_variables].describe

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(df.loc[:,highly_correlated_variables],df.loc[:,"y"])

df_test = pd.read_csv("../input/test.csv")
res =  rf.predict(df_test.loc[:,highly_correlated_variables])

In [ ]:
df_test.loc[2001,"ID"]

In [ ]:
out = open('output_1.csv', "w")
out.write("ID,y\n")

for i in range(0,res.shape[0]):
    out.write(str(df_test.loc[i,"ID"])+","+str(res[i])+"\n")
              
out.close()
    